<a href="https://colab.research.google.com/github/lpaulette/CSE150A-group_project/blob/main/SuicideWatch_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import os

In [ ]:
# Download the dataset
path = kagglehub.dataset_download("asinow/schizohealth-dataset")
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/asinow/schizohealth-dataset/versions/1


In [ ]:
# Locate CSV file (modify if the dataset contains multiple files)
csv_file = [f for f in os.listdir(path) if f.endswith(".csv")][0]  # Get first CSV file
data_path = os.path.join(path, csv_file)

In [ ]:
# Load dataset
df = pd.read_csv(data_path)

In [ ]:
df.head()

,Hasta_ID,Yaş,Cinsiyet,Eğitim_Seviyesi,Medeni_Durum,Meslek,Gelir_Düzeyi,Yaşadığı_Yer,Tanı,Hastalık_Süresi,Hastaneye_Yatış_Sayısı,Ailede_Şizofreni_Öyküsü,Madde_Kullanımı,İntihar_Girişimi,Pozitif_Semptom_Skoru,Negatif_Semptom_Skoru,GAF_Skoru,Sosyal_Destek,Stres_Faktörleri,İlaç_Uyumu
0,1,72,1,4,2,0,2,1,0,0,0,0,0,0,32,48,72,0,2,2
1,2,49,1,5,2,2,1,0,1,35,1,1,1,1,51,63,40,2,2,0
2,3,53,1,5,3,2,1,0,1,32,0,1,0,0,72,85,51,0,1,1
3,4,67,1,3,2,0,2,0,0,0,0,0,1,0,10,21,74,1,1,2
4,5,54,0,1,2,0,2,1,0,0,0,0,0,0,4,27,98,0,1,0


In [ ]:
column_mapping = {
    "Hasta_ID" : "Patient ID",
    "Yaş": "Age",
    "Cinsiyet": "Gender",
    "Eğitim_Seviyesi" : "Education_level",
    "Medeni_Durum" : "Marital_status",
    "Meslek" : "Occupation",
    "Gelir_Düzeyi": "Income_level",
    "Yaşadığı_Yer" : "Living_area",
    "Tanı": "Diagnosis",
    "Hastalık_Süresi": "Disease_duration",
    "Madde_Kullanımı": "Substance_use",
    "Hastaneye_Yatış_Sayısı": "Hospitalizations",
    "Ailede_Şizofreni_Öyküsü": "Family_history",
    "Stres_Faktörleri": "Stress_factors",
    "Sosyal_Destek": "Social_support",
    "İntihar_Girişimi": "Suicide_attempt",
    "Pozitif_Semptom_Skoru" : "Positive Symptom Score",
    "Negatif_Semptom_Skoru" : "Negative Symptom Score",
    "İlaç_Uyumu": "Medication_adherence"
}

# Rename the columns in the DataFrame
df = df.rename(columns=column_mapping)
df.head()

,Patient ID,Age,Gender,Education_level,Marital_status,Occupation,Income_level,Living_area,Diagnosis,Disease_duration,Hospitalizations,Family_history,Substance_use,Suicide_attempt,Positive Symptom Score,Negative Symptom Score,GAF_Skoru,Social_support,Stress_factors,Medication_adherence
0,1,72,1,4,2,0,2,1,0,0,0,0,0,0,32,48,72,0,2,2
1,2,49,1,5,2,2,1,0,1,35,1,1,1,1,51,63,40,2,2,0
2,3,53,1,5,3,2,1,0,1,32,0,1,0,0,72,85,51,0,1,1
3,4,67,1,3,2,0,2,0,0,0,0,0,1,0,10,21,74,1,1,2
4,5,54,0,1,2,0,2,1,0,0,0,0,0,0,4,27,98,0,1,0


In [ ]:
# Define a function to assign binary categories
def categorize_age(age):
    return pd.Series({
        'age_18_30': int(age < 31),
        'age_31_50': int(31 <= age < 51),
        'age_51_80': int(age >= 51)
    })

def categorize_education(education):
    return pd.Series({
        'primary': int(education == 1),
        'middle_school': int(education == 2),
        'highschool': int(education == 3),
        'university': int(education == 4),
        'postgrad': int(education == 5)
    })

def categorize_marital_status(status):
    return pd.Series({
        'single': int(status == 0),
        'married': int(status == 1),
        'divorced': int(status == 2),
        'widowed': int(status == 3),

    })

def categorize_occupation(occupation):
    return pd.Series({
        'unemployed': int(occupation == 0),
        'employed': int(occupation == 1),
        'retired': int(occupation == 2),
        'student': int(occupation == 3),

    })

def categorize_income(income):
    return pd.Series({
        'low_income': int(income == 0)
    })

def categorize_disease_duration(duration):
    return pd.Series({
        '0_to_5_with_disease': int(duration < 6),
        '6_to_15_with_disease': int(duration < 16 and duration > 5),
        '16_to_30_with_disease': int(duration < 31 and duration > 15),
        '31_to_40_with_disease': int(duration > 30)
    })

def categorize_hospitalizations(hospitalizations):
    return pd.Series({
        'less_than_5_hospitalizations': int(hospitalizations < 6),
        'more_than_5_hospitalizations': int(hospitalizations > 5)
    })

def categorize_medication_adherence(medication_adherence):
    return pd.Series({
        'low_medication_adherence': int(medication_adherence == 0)
    })

def categorize_social_support(social_support):
    return pd.Series({
        'Social_support': int(social_support != 0)
    })

def categorize_stress_factors(stress_factors):
  return pd.Series({
    'High_stress': int(stress_factors == 2)
  })

In [ ]:
# Apply the function to create new binary columns
df[['age_18_30', 'age_31_50', 'age_51_80']] = df['Age'].apply(categorize_age)
df[['primary', 'middle_school', 'highschool', 'university', 'postgrad']] = df['Education_level'].apply(categorize_education)
df[['single', 'married', 'divorced', 'widowed']] = df['Marital_status'].apply(categorize_marital_status)
df[['unemployed', 'employed', 'retired', 'student']] = df['Occupation'].apply(categorize_occupation)
df[['low_income']] = df['Income_level'].apply(categorize_income)
df[['0_to_5_with_disease', '6_to_15_with_disease', '16_to_30_with_disease', '31_to_40_with_disease']] = df['Disease_duration'].apply(categorize_disease_duration)
df[['less_than_5_hospitalizations', 'more_than_5_hospitalizations']] = df['Hospitalizations'].apply(categorize_hospitalizations)
df[['low_medication_adherence']] = df['Medication_adherence'].apply(categorize_medication_adherence)
df[['Social_support']] = df['Social_support'].apply(categorize_social_support)
df[['High_stress']] = df['Stress_factors'].apply(categorize_stress_factors)

# Drop the original 'Age' column if no longer needed
df.drop(columns=['Age', 'Education_level', 'Marital_status', 'Occupation', 'Income_level', 'Disease_duration',
                 'Hospitalizations', 'Medication_adherence', 'Positive Symptom Score', 'Negative Symptom Score',
                 'GAF_Skoru', 'Stress_factors'], inplace=True)

df.head()



,Patient ID,Gender,Living_area,Diagnosis,Family_history,Substance_use,Suicide_attempt,Social_support,age_18_30,age_31_50,...,student,low_income,0_to_5_with_disease,6_to_15_with_disease,16_to_30_with_disease,31_to_40_with_disease,less_than_5_hospitalizations,more_than_5_hospitalizations,low_medication_adherence,High_stress
0,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1,2,1,0,1,1,1,1,1,0,1,...,0,0,0,0,0,1,1,0,1,1
2,3,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,4,1,0,0,0,1,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
4,5,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0


In [ ]:
# Define Noisy OR likelihood function
# [p_vals] : Array list of probabilites
# [X] : Matrix of input features
# [y] : Target variable (suicide attempt)

def noisy_or_likelihood(p_vals, X, y):
    # Clip p_vals to a small value above 0 and below 1 to avoid numerical instability
    # @@ replace this, we'll get p_vals with EM algorithm

    X_values = np.array(list(X.values()))
    p_values = np.array([p_vals[feature] for feature in X.keys()])

    # Clip probs as well to avoid log(0) and log(1) errors
    # @@ not sure we need this
    # p_values = np.clip(p_vals, 1e-6, 1 - 1e-6)

    # @@ X is given, i.e.: {schizophrenia = 1, substance_abuse = 0, ...}
    prob = 1 - np.prod((1 - p_values) ** X_values)

    # @@ not sure we need this...
    # @@ just need P(Y=1 | X1, ..., Xn) to determine if patient should be put on suicide watch
    # log_likelihood = np.sum(y * np.log(prob) + (1 - y) * np.log(1 - prob))
    # return -log_likelihood  # Negative because we minimize
    return prob

In [ ]:
# Estimate Noisy OR parameters
# [data] : Data frame containing pur values
# [features]: The column names that represent our independt variables
# [target]: The column that represents our dependent variable (suicide attempt)

def estimate_noisy_or_probs(data, features, target):

   # @@ Use EM algorithm to calculate p_values, done :D

    #Get the feature matrix and the target variable
    X = data[features].values
    # print("---X---")
    # print(X)
    y = data[target].values
    n_samples, n_features = X.shape

    #Initialize probabilites randomly
    p_vals = np.random.uniform(0.1, 0.9, size = n_features)

    max_iter = 100
    tol=1e-6

    for iteration in range(max_iter):
      old_p_vals = p_vals.copy()

      #E-step: Compute posterior probabilties of latent causes
      # Calculate probabilities for each data point
      prob_yGivenx = 1 - np.prod((1 - p_vals) ** X, axis = 1)
      prob_yGivenx = np.clip(prob_yGivenx, 1e-6, 1 - 1e-6) #Clip to avoid log(0) and log(1) errors

      responsibilities = (y[: , None] * X) / prob_yGivenx[:, None] #Calculate responsibilities for each feature

      #M-step: pdate probability estimates
      p_vals = np.sum(responsibilities, axis = 0) / np.sum(X, axis = 0)

      # Check for convergence
      if np.linalg.norm(p_vals - old_p_vals) < tol:
        break


    return dict(zip(features, p_vals))

In [ ]:
# Variables we want to consider
columns_list = df.columns.to_list()
if 'Patient ID' in columns_list:
    columns_list.remove('Patient ID')

print(columns_list)

features = columns_list
target = "Suicide_attempt"

['Gender', 'Living_area', 'Diagnosis', 'Family_history', 'Substance_use', 'Suicide_attempt', 'Social_support', 'age_18_30', 'age_31_50', 'age_51_80', 'primary', 'middle_school', 'highschool', 'university', 'postgrad', 'single', 'married', 'divorced', 'widowed', 'unemployed', 'employed', 'retired', 'student', 'low_income', '0_to_5_with_disease', '6_to_15_with_disease', '16_to_30_with_disease', '31_to_40_with_disease', 'less_than_5_hospitalizations', 'more_than_5_hospitalizations', 'low_medication_adherence', 'High_stress']


In [ ]:
# Estimate probabilities
p_estimates = estimate_noisy_or_probs(df, features, target)

---X---
[[1 1 0 ... 0 0 1]
 [1 0 1 ... 0 1 1]
 [1 0 1 ... 0 0 0]
 ...
 [1 0 1 ... 1 0 0]
 [1 1 0 ... 0 0 0]
 [0 1 0 ... 0 1 1]]


In [ ]:
# Print results
print("Estimated probabilities for Noisy OR model:")
for factor, p in p_estimates.items():
    print(f"{factor}: {p:.4f}")

Estimated probabilities for Noisy OR model:
Gender: 0.0901
Living_area: 0.0827
Diagnosis: 0.3062
Family_history: 0.1668
Substance_use: 0.1571
Suicide_attempt: 1.0000
Social_support: 0.0870
age_18_30: 0.0795
age_31_50: 0.0887
age_51_80: 0.0920
primary: 0.0844
middle_school: 0.1003
highschool: 0.0817
university: 0.0902
postgrad: 0.0859
single: 0.0870
married: 0.0890
divorced: 0.0956
widowed: 0.0821
unemployed: 0.0803
employed: 0.0998
retired: 0.0831
student: 0.0898
low_income: 0.0882
0_to_5_with_disease: 0.0167
6_to_15_with_disease: 0.3362
16_to_30_with_disease: 0.2903
31_to_40_with_disease: 0.2925
less_than_5_hospitalizations: 0.0562
more_than_5_hospitalizations: 0.3000
low_medication_adherence: 0.1658
High_stress: 0.0891


In [ ]:
# @@ Decide whether to place patient in suicide watch

# drop 'Suicide_attempt' from p_estimates  ->  type: dict(zip(features, p_vals))
if "Suicide_attempt" in p_estimates:
    del p_estimates["Suicide_attempt"]

# select features from X  ->  type: user-assigned {0, 1} for each feature we want
patient_history = {'Gender': 1, # Woman: 0, Man : 1
                            'Living_area': 1, # Urban
                            'Diagnosis': 1, # with schizophrenia
                            'Family_history': 1, #with genetic predisposition
                            'Substance_use': 1, # yes
                            # 'Suicide_attempt':,
                            'Social_support': 1, # yes
                            'High_stress': 0, # yes
                            'age_18_30': 0,
                            'age_31_50': 1,
                            'age_51_80': 0,
                            'primary': 0,
                            'middle_school': 0,
                            'highschool': 0,
                            'university': 1,
                            'postgrad': 1,
                            'single': 0,
                            'married': 0,
                            'divorced': 0,
                            'widowed': 1,
                            'unemployed': 1,
                            'employed': 0,
                            'retired': 0,
                            'student': 0,
                            'low_income': 0,
                            '0_to_5_with_disease': 1,
                            '6_to_15_with_disease': 0,
                            '16_to_30_with_disease': 0,
                            '31_to_40_with_disease': 0,
                            'less_than_5_hospitalizations': 1,
                            'more_than_5_hospitalizations': 0,
                            'low_medication_adherence': 0
}
# use noisy_or_likelihood(p_vals, X, y)

noisy_OR_prob = noisy_or_likelihood(p_estimates, patient_history, 'Suicide_attempt')
print("noisy OR prob: ", noisy_OR_prob)


if(noisy_OR_prob * 100 > 60):
  print("Patient should be put on suicide watch")
else:
  print("Patient should not be put on suicide watch")

noisy OR prob:  0.7795514701018064
Patient should be put on suicide watch


# Testing
We calcuate the probability of a patient attempting suicide based on gender only.

In [ ]:
def test(
  df: pd.DataFrame,
  patient_history: dict[str, int],
  feature_list: list[str]
   ):

  p_vals = estimate_noisy_or_probs(df, feature_list, 'Suicide_attempt')

  # drop features that we're not using
  for ft in feature_list:
    if ft not in patient_history.keys():
      del p_vals[ft]

  # print(p_vals.keys())

  noisy_OR_prob = noisy_or_likelihood(p_estimates, patient_history, 'Suicide_attempt')
  print(f"Probability of a suicide attempt: {noisy_OR_prob:.4f}")

  if(noisy_OR_prob * 100 > 60):
    print("Patient should be put on suicide watch")
  else:
    print("Patient should not be put on suicide watch")


# Gender



In [ ]:
patient_history = {'Gender': 1, # Woman: 0, Man: 1
                   'Diagnosis': 1
}

print("----Men with schizophrenia----")

test(df, patient_history, columns_list)

patient_history = {'Gender': 1, # Woman: 0, Man: 1
                   'Diagnosis': 0
}

print("----Men without schizophrenia----")

test(df, patient_history, columns_list)

patient_history = {'Gender': 0, # Woman: 0, Man: 1
                   'Diagnosis': 1
}

print("----Women with schizophrenia----")

test(df, patient_history, columns_list)

patient_history = {'Gender': 0, # Woman: 0, Man: 1
                   'Diagnosis': 0
}

print("----Women without schizophrenia----")

test(df, patient_history, columns_list)

----Men with schizophrenia----
Probability of a suicide attempt: 0.3687
Patient should not be put on suicide watch
----Men without schizophrenia----
Probability of a suicide attempt: 0.0901
Patient should not be put on suicide watch
----Women with schizophrenia----
Probability of a suicide attempt: 0.3062
Patient should not be put on suicide watch
----Women without schizophrenia----
Probability of a suicide attempt: 0.0000
Patient should not be put on suicide watch


# Marital Status

In [ ]:
print("----Men with schizophrenia----")

print("single")
patient_history = {'Gender': 1,
                    'Diagnosis': 1,
                     'single': 1,
                     'married': 0,
                     'divorced': 0,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("married")
patient_history = {'Gender': 1,
                    'Diagnosis': 1,
                     'single': 0,
                     'married': 1,
                     'divorced': 0,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("divorced")
patient_history = {'Gender': 1,
                    'Diagnosis': 1,
                     'single': 0,
                     'married': 0,
                     'divorced': 1,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("widowed")
patient_history = {'Gender': 1,
                    'Diagnosis': 1,
                     'single': 0,
                     'married': 0,
                     'divorced': 0,
                     'widowed': 1
  }
test(df, patient_history, columns_list)

----Men with schizophrenia----
single
Probability of a suicide attempt: 0.4237
Patient should not be put on suicide watch
married
Probability of a suicide attempt: 0.4249
Patient should not be put on suicide watch
divorced
Probability of a suicide attempt: 0.4291
Patient should not be put on suicide watch
widowed
Probability of a suicide attempt: 0.4206
Patient should not be put on suicide watch


In [ ]:
print("----Men without schizophrenia----")

print("single")
patient_history = {'Gender': 1,
                    'Diagnosis': 0,
                     'single': 1,
                     'married': 0,
                     'divorced': 0,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("married")
patient_history = {'Gender': 1,
                    'Diagnosis': 0,
                     'single': 0,
                     'married': 1,
                     'divorced': 0,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("divorced")
patient_history = {'Gender': 1,
                    'Diagnosis': 0,
                     'single': 0,
                     'married': 0,
                     'divorced': 1,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("widowed")
patient_history = {'Gender': 1,
                    'Diagnosis': 0,
                     'single': 0,
                     'married': 0,
                     'divorced': 0,
                     'widowed': 1
  }
test(df, patient_history, columns_list)

----Men without schizophrenia----
single
Probability of a suicide attempt: 0.1693
Patient should not be put on suicide watch
married
Probability of a suicide attempt: 0.1711
Patient should not be put on suicide watch
divorced
Probability of a suicide attempt: 0.1771
Patient should not be put on suicide watch
widowed
Probability of a suicide attempt: 0.1649
Patient should not be put on suicide watch


In [ ]:
print("----Women with schizophrenia----")

print("single")
patient_history = {'Gender': 0,
                    'Diagnosis': 1,
                     'single': 1,
                     'married': 0,
                     'divorced': 0,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("married")
patient_history = {'Gender': 0,
                    'Diagnosis': 1,
                     'single': 0,
                     'married': 1,
                     'divorced': 0,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("divorced")
patient_history = {'Gender': 0,
                    'Diagnosis': 1,
                     'single': 0,
                     'married': 0,
                     'divorced': 1,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("widowed")
patient_history = {'Gender': 0,
                    'Diagnosis': 1,
                     'single': 0,
                     'married': 0,
                     'divorced': 0,
                     'widowed': 1
  }
test(df, patient_history, columns_list)

----Women with schizophrenia----
single
Probability of a suicide attempt: 0.3666
Patient should not be put on suicide watch
married
Probability of a suicide attempt: 0.3680
Patient should not be put on suicide watch
divorced
Probability of a suicide attempt: 0.3725
Patient should not be put on suicide watch
widowed
Probability of a suicide attempt: 0.3632
Patient should not be put on suicide watch


In [ ]:
print("----Women without schizophrenia----")

print("single")
patient_history = {'Gender': 0,
                    'Diagnosis': 0,
                     'single': 1,
                     'married': 0,
                     'divorced': 0,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("married")
patient_history = {'Gender': 0,
                    'Diagnosis': 0,
                     'single': 0,
                     'married': 1,
                     'divorced': 0,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("divorced")
patient_history = {'Gender': 0,
                    'Diagnosis': 0,
                     'single': 0,
                     'married': 0,
                     'divorced': 1,
                     'widowed': 0
  }
test(df, patient_history, columns_list)

print("widowed")
patient_history = {'Gender': 0,
                    'Diagnosis': 0,
                     'single': 0,
                     'married': 0,
                     'divorced': 0,
                     'widowed': 1
  }
test(df, patient_history, columns_list)

----Women without schizophrenia----
single
Probability of a suicide attempt: 0.0870
Patient should not be put on suicide watch
married
Probability of a suicide attempt: 0.0890
Patient should not be put on suicide watch
divorced
Probability of a suicide attempt: 0.0956
Patient should not be put on suicide watch
widowed
Probability of a suicide attempt: 0.0821
Patient should not be put on suicide watch
